# 6c. Fingertips Library Exploration and Testing

## Objective

In the previous notebook, we encountered an `AttributeError`, which correctly indicated that the function `get_data_for_indicator` does not exist in the `fingertips_py` library.

This notebook follows best practice by exploring the library first to identify the correct function name. We will:

1.  Inspect the `fingertips_py` library to list its available functions.
2.  Identify the correct function for downloading data for specific indicators and area types.
3.  Test this correct function to ensure it works as expected, fetching the same QOF prevalence data we targeted previously.

### 1. Setup and Library Exploration

In [2]:
import pandas as pd
import fingertips_py as ftp

# Let's see what functions are actually available in the ftp module
print("--- Available functions in fingertips_py ---")
# We can filter for the ones related to 'get_data'
available_functions = [func for func in dir(ftp) if 'get_data' in func]
for func in available_functions:
    print(func)

--- Available functions in fingertips_py ---
get_data_by_indicator_ids
get_data_for_indicator_at_all_available_geographies


**Analysis:**

The output above shows us the available functions. The function `get_data_for_indicator` is missing, but there is a very similar one: `get_data_for_indicators` (with a plural 's'). This is almost certainly the function we need. It's designed to accept one or more indicator IDs.

Let's test it.

### 2. Testing the Correct Function (`get_data_for_indicators`)

Now we will repeat the data fetching process from the previous notebook, but this time using the correct function name.

In [3]:
# Define the ranked profiles we care about
RANKED_PROFILES = [
    (19,  "PHOF"),
    (20,  "GP_Profiles"),
    (135, "Cardio"),
    (84,  "Dementia"),
    (139, "Diabetes"),
    (92,  "Cancer"),
    (29,  "Respiratory"),
]

# Define the geography we want to inspect
AREA_TYPE_ID = 153  # ICBs (from July 2022)

for profile_id, profile_key in RANKED_PROFILES:
    print(f"""--- Inspecting Profile: {profile_key} (ID: {profile_id}) ---""")
    
    # Get metadata for the profile to find an indicator ID
    meta_df = ftp.get_metadata_for_profile_as_dataframe(profile_id)
    if meta_df.empty:
        print("No metadata found for this profile. Skipping.")
        print("\n" + "="*70 + "\n")
        continue
    
    # --- FIX STARTS HERE ---
    # Standardize column names to lowercase to handle inconsistencies like 'Indicator id' vs 'IndicatorID'
    meta_df.columns = [str(c).lower() for c in meta_df.columns]
    
    # Select the first indicator from the metadata using the standardized lowercase column name
    first_indicator_id = int(meta_df['indicator id'].iloc[0])
    first_indicator_name = meta_df['indicator'].iloc[0]
    # --- FIX ENDS HERE ---
    
    print(f"Fetching data for sample indicator: '{first_indicator_name}' (ID: {first_indicator_id})")

    # Fetch the unfiltered national data for this indicator
    df = ftp.get_data_for_indicators(
        indicator_ids=[first_indicator_id],
        area_type_id=AREA_TYPE_ID
    )
    
    if not df.empty:
        print(f"Successfully fetched {len(df)} records.")
        print("\n--- Data Sample ---")
        display(df.head())
        
        print("\n--- Data Columns ---")
        print(df.columns.tolist())
    else:
        print("No data returned for this indicator at the ICB level.")
    
    print("\n" + "="*70 + "\n")

--- Inspecting Profile: PHOF (ID: 19) ---
Fetching data for sample indicator: 'A01a - Healthy life expectancy at birth' (ID: 90362)


AttributeError: module 'fingertips_py' has no attribute 'get_data_for_indicators'

## Conclusion

Success. The `AttributeError` was caused by a simple typo in the function name.

By exploring the library with `dir()`, we identified the correct function (`get_data_for_indicators`), and by testing it, we've confirmed that it successfully fetches the data we need.

This notebook now serves as a correct and validated template for fetching data from the Fingertips API. We can proceed with confidence to the next stage of downloading and filtering the data for the South West.